In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

## In this regression model, estimated salary is the output, and the remaining features are input

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data= data.drop(['RowNumber','CustomerId','Surname'], axis=1)

In [4]:
# Label encode "Gender"
label_encoder_gender= LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [5]:
# One-hot encode "Geography"
oneHotencode_geo= OneHotEncoder()
geo_encoded= oneHotencode_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df= pd.DataFrame(geo_encoded, columns=oneHotencode_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [6]:
# COmbine the data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [7]:
## Split the data
X= data.drop('EstimatedSalary', axis=1)
y= data['EstimatedSalary']

X_train,X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)


In [8]:
# Scaling 

scaler_reg=StandardScaler()
X_train=scaler_reg.fit_transform(X_train)
X_test= scaler_reg.transform(X_test)

In [9]:
# SAVE THE PICKLE FILES

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('oneHotencode_geo.pkl','wb') as file:
    pickle.dump(oneHotencode_geo, file)

with open('scaler_reg.pkl','wb') as file:
    pickle.dump(scaler_reg, file)

ANN regression model implementation

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


In [11]:
# Build the model
model= Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for estimated salary
])
# In the output layer, no activation function is applies because
# there is already a default fucntion called linear activation function
# for regression problems. This means we will use the output as it is.

In [12]:
# Compile the model

model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# Setup tensorboard and early stopping

log_dir= 'regressionlogs/fit/'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cbk= TensorBoard(log_dir=log_dir, histogram_freq=1)

early_stop_cbk=EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [14]:
# Train model

history=model.fit(
    X_train, y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[tensorboard_cbk, early_stop_cbk]
)

Epoch 1/100


250/250 [==============================] - 2s 3ms/step - loss: 100370.0703 - mae: 100370.0703 - val_loss: 98491.5703 - val_mae: 98491.5703
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 99537.0625 - mae: 99537.0625 - val_loss: 96798.6562 - val_mae: 96798.6562
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 96567.4453 - mae: 96567.4453 - val_loss: 92428.4141 - val_mae: 92428.4141
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 90756.0078 - mae: 90756.0078 - val_loss: 85266.0312 - val_mae: 85266.0312
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 82412.9688 - mae: 82412.9688 - val_loss: 76192.0625 - val_mae: 76192.0625
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 72840.6562 - mae: 72840.6562 - val_loss: 67142.3203 - val_mae: 67142.3203
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 63922.1289 - mae: 63922.128

In [15]:
%load_ext tensorboard

In [16]:
%tensorboard --logdir regressionlogs/fit/

Reusing TensorBoard on port 6007 (pid 729536), started 3 days, 23:29:44 ago. (Use '!kill 729536' to kill it.)

In [17]:
## Evaluate the model
test_loss, test_mae = model.evaluate(X_test,y_test)
print(f"Test MAE :{test_mae}")

63/63 [==============================] - 0s 1ms/step - loss: 50353.8086 - mae: 50353.8086
Test MAE :50353.80859375


In [18]:
# Save the regression model
model.save("Regression_model.h5")

c:\Users\aasth\UDEMY PROJECTS\GEN_AI_KRISHNAIK\ANN_ChurnPredictionn\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
